# Ex - GroupBy

### Introduction:

GroupBy can be summarized as Split-Apply-Combine.

Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

Check out this [Diagram](http://i.imgur.com/yjNkiwL.png)  
### Step 1. Import the necessary libraries

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import requests

In [3]:
spark = SparkSession.builder.master("local[1]").appName('drinks').getOrCreate()

22/09/07 15:09:17 WARN Utils: Your hostname, xkeyscore resolves to a loopback address: 127.0.1.1; using 192.168.1.8 instead (on interface wlp0s20f3)
22/09/07 15:09:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/07 15:09:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv). 

In [3]:
url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv"
data_content = requests.get(url)
with open("drinks.csv", "w") as f:
    f.write(data_content.text)

### Step 3. Assign it to a variable called drinks.

In [4]:
drinks = spark.read.options(header=True, inferSchema=True).csv("drinks.csv")

In [5]:
drinks.columns

['country',
 'beer_servings',
 'spirit_servings',
 'wine_servings',
 'total_litres_of_pure_alcohol',
 'continent']

In [6]:
drinks.show(5)

+-----------+-------------+---------------+-------------+----------------------------+---------+
|    country|beer_servings|spirit_servings|wine_servings|total_litres_of_pure_alcohol|continent|
+-----------+-------------+---------------+-------------+----------------------------+---------+
|Afghanistan|            0|              0|            0|                         0.0|       AS|
|    Albania|           89|            132|           54|                         4.9|       EU|
|    Algeria|           25|              0|           14|                         0.7|       AF|
|    Andorra|          245|            138|          312|                        12.4|       EU|
|     Angola|          217|             57|           45|                         5.9|       AF|
+-----------+-------------+---------------+-------------+----------------------------+---------+
only showing top 5 rows



### Step 4. Which continent drinks more beer on average?

In [7]:
drinks.select("continent", "beer_servings").groupBy("continent").avg().show()

+---------+------------------+
|continent|avg(beer_servings)|
+---------+------------------+
|       NA|145.43478260869566|
|       SA|175.08333333333334|
|       AS| 37.04545454545455|
|       OC|           89.6875|
|       EU|193.77777777777777|
|       AF|61.471698113207545|
+---------+------------------+



### Step 5. For each continent print the statistics for wine consumption.

In [8]:
# there is no direct method of calculating summary per country. We could work on creating multiple aggregations using the .agg() function

In [9]:
drinks.summary().show()

+-------+-----------+------------------+-----------------+-----------------+----------------------------+---------+
|summary|    country|     beer_servings|  spirit_servings|    wine_servings|total_litres_of_pure_alcohol|continent|
+-------+-----------+------------------+-----------------+-----------------+----------------------------+---------+
|  count|        193|               193|              193|              193|                         193|      193|
|   mean|       null|106.16062176165804|80.99481865284974|49.45077720207254|           4.717098445595855|     null|
| stddev|       null| 101.1431025393134|88.28431210968618|79.69759845763012|           3.773298164356082|     null|
|    min|Afghanistan|                 0|                0|                0|                         0.0|       AF|
|    25%|       null|                20|                4|                1|                         1.3|     null|
|    50%|       null|                76|               56|              

In [16]:
drinks.select("*").groupby("continent")\
            .agg(
                    avg("beer_servings"),avg("spirit_servings"), avg("wine_servings"), avg("total_litres_of_pure_alcohol"), 
                    min("beer_servings"),min("spirit_servings"), min("wine_servings"), min("total_litres_of_pure_alcohol"),
                    max("beer_servings"),max("spirit_servings"), max("wine_servings"), max("total_litres_of_pure_alcohol"),
).show()

+---------+------------------+--------------------+------------------+---------------------------------+------------------+--------------------+------------------+---------------------------------+------------------+--------------------+------------------+---------------------------------+
|continent|avg(beer_servings)|avg(spirit_servings)|avg(wine_servings)|avg(total_litres_of_pure_alcohol)|min(beer_servings)|min(spirit_servings)|min(wine_servings)|min(total_litres_of_pure_alcohol)|max(beer_servings)|max(spirit_servings)|max(wine_servings)|max(total_litres_of_pure_alcohol)|
+---------+------------------+--------------------+------------------+---------------------------------+------------------+--------------------+------------------+---------------------------------+------------------+--------------------+------------------+---------------------------------+
|       NA|145.43478260869566|   165.7391304347826| 24.52173913043478|                5.995652173913044|                 1|    

### Step 6. Print the mean alcohol consumption per continent for every column

In [23]:
drinks.groupby("continent").agg(
    mean("total_litres_of_pure_alcohol").alias("total_litres_of_pure_alcohol"),
    mean("beer_servings").alias("beer_servings"),
    mean("spirit_servings").alias("spirit_servings"),
    mean("wine_servings").alias("wine_servings")
).show(10)

+---------+----------------------------+------------------+------------------+------------------+
|continent|total_litres_of_pure_alcohol|     beer_servings|   spirit_servings|     wine_servings|
+---------+----------------------------+------------------+------------------+------------------+
|       NA|           5.995652173913044|145.43478260869566| 165.7391304347826| 24.52173913043478|
|       SA|           6.308333333333334|175.08333333333334|            114.75|62.416666666666664|
|       AS|          2.1704545454545454| 37.04545454545455| 60.84090909090909| 9.068181818181818|
|       OC|          3.3812500000000005|           89.6875|           58.4375|            35.625|
|       EU|           8.617777777777777|193.77777777777777|132.55555555555554|142.22222222222223|
|       AF|            3.00754716981132|61.471698113207545|16.339622641509433|16.264150943396228|
+---------+----------------------------+------------------+------------------+------------------+



### Step 7. Print the median alcohol consumption per continent for every column

In [27]:
drinks.groupby("continent").agg(
    percentile_approx("total_litres_of_pure_alcohol",0.5).alias("total_litres_of_pure_alcohol"),
    percentile_approx("beer_servings",0.5).alias("beer_servings"),
    percentile_approx("spirit_servings",0.5).alias("spirit_servings"),
    percentile_approx("wine_servings",0.5).alias("wine_servings")
).show(10)

+---------+----------------------------+-------------+---------------+-------------+
|continent|total_litres_of_pure_alcohol|beer_servings|spirit_servings|wine_servings|
+---------+----------------------------+-------------+---------------+-------------+
|       NA|                         6.3|          143|            137|           11|
|       SA|                         6.6|          162|            100|            8|
|       AS|                         1.0|           16|             16|            1|
|       OC|                         1.5|           49|             35|            8|
|       EU|                        10.0|          219|            122|          128|
|       AF|                         2.3|           32|              3|            2|
+---------+----------------------------+-------------+---------------+-------------+



### Step 8. Print the mean, min and max values for spirit consumption.
#### This time output a DataFrame

In [29]:
drinks.groupby("continent").agg(
    mean("total_litres_of_pure_alcohol").alias("mean_total_litres_of_pure_alcohol"),
    min("total_litres_of_pure_alcohol").alias("min_total_litres_of_pure_alcohol"),
    max("total_litres_of_pure_alcohol").alias("max_mean_total_litres_of_pure_alcohol"),
    
).show(10)

+---------+---------------------------------+--------------------------------+-------------------------------------+
|continent|mean_total_litres_of_pure_alcohol|min_total_litres_of_pure_alcohol|max_mean_total_litres_of_pure_alcohol|
+---------+---------------------------------+--------------------------------+-------------------------------------+
|       NA|                5.995652173913044|                             2.2|                                 11.9|
|       SA|                6.308333333333334|                             3.8|                                  8.3|
|       AS|               2.1704545454545454|                             0.0|                                 11.5|
|       OC|               3.3812500000000005|                             0.0|                                 10.4|
|       EU|                8.617777777777777|                             0.0|                                 14.4|
|       AF|                 3.00754716981132|                   